In [1]:
import mlflow
import pandas as pd
import mlflow.sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import numpy as np

In [28]:
df = pd.read_csv(
    "https://raw.githubusercontent.com/campusx-official/jupyter-masterclass/main/tweet_emotions.csv"
).drop(columns=["tweet_id"])
df.head()

,sentiment,content
0,empty,@tiffanylue i know i was listenin to bad habi...
1,sadness,Layin n bed with a headache ughhhh...waitin o...
2,sadness,Funeral ceremony...gloomy friday...
3,enthusiasm,wants to hang out with friends SOON!
4,neutral,@dannycastillo We want to trade with someone w...


### Data preprocessing

In [29]:
# Define text preprocessing functions
def lemmatization(text):
    """Lemmatize the text."""
    lemmatizer = WordNetLemmatizer()
    text = text.split()
    text = [lemmatizer.lemmatize(word) for word in text]
    return " ".join(text)


def remove_stop_words(text):
    """Remove stop words from the text."""
    stop_words = set(stopwords.words("english"))
    text = [word for word in str(text).split() if word not in stop_words]
    return " ".join(text)


def removing_numbers(text):
    """Remove numbers from the text."""
    text = "".join([char for char in text if not char.isdigit()])
    return text


def lower_case(text):
    """Convert text to lower case."""
    text = text.split()
    text = [word.lower() for word in text]
    return " ".join(text)


def removing_punctuations(text):
    """Remove punctuations from the text."""
    text = re.sub("[%s]" % re.escape(string.punctuation), " ", text)
    text = text.replace("؛", "")
    text = re.sub("\s+", " ", text).strip()
    return text


def removing_urls(text):
    """Remove URLs from the text."""
    url_pattern = re.compile(r"https?://\S+|www\.\S+")
    return url_pattern.sub(r"", text)


def normalize_text(df):
    """Normalize the text data."""
    try:
        df.loc[:, "content"] = df.loc[:, "content"].apply(lower_case)
        df.loc[:, "content"] = df.loc[:, "content"].apply(remove_stop_words)
        df.loc[:, "content"] = df.loc[:, "content"].apply(removing_numbers)
        df.loc[:, "content"] = df.loc[:, "content"].apply(removing_punctuations)
        df.loc[:, "content"] = df.loc[:, "content"].apply(removing_urls)
        df.loc[:, "content"] = df.loc[:, "content"].apply(lemmatization)
        return df
    except Exception as e:
        print(f"Error during text normalization: {e}")
        raise

In [30]:
x = df.loc[:, "sentiment"].isin(["happiness", "sadness"])
df = df[x]

In [31]:
df = normalize_text(df)
df.head()

,sentiment,content
1,sadness,layin n bed headache ughhhh waitin call
2,sadness,funeral ceremony gloomy friday
6,sadness,sleep im not thinking old friend want he s mar...
8,sadness,charviray charlene love miss
9,sadness,kelcouch i m sorry least friday


In [32]:
df.sentiment.value_counts()

sentiment
happiness    5209
sadness      5165
Name: count, dtype: int64

In [33]:
df.loc[:, "sentiment"] = df.loc[:, "sentiment"].map({"sadness": 0, "happiness": 1}) 
df.head()

,sentiment,content
1,0,layin n bed headache ughhhh waitin call
2,0,funeral ceremony gloomy friday
6,0,sleep im not thinking old friend want he s mar...
8,0,charviray charlene love miss
9,0,kelcouch i m sorry least friday


In [34]:
df

,sentiment,content
1,0,layin n bed headache ughhhh waitin call
2,0,funeral ceremony gloomy friday
6,0,sleep im not thinking old friend want he s mar...
8,0,charviray charlene love miss
9,0,kelcouch i m sorry least friday
...,...,...
39986,1,going watch boy striped pj s hope cry
39987,1,gave bike thorough wash degrease grease it thi...
39988,1,amazing time last night mcfly incredible
39994,1,succesfully following tayla


In [35]:
vectorizer = CountVectorizer(max_features=5000)
X = vectorizer.fit_transform(df["content"])
y = df["sentiment"]

In [36]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [37]:
y_train

23531    0
8051     0
11499    0
31288    1
18561    0
        ..
21697    1
19445    0
20216    1
3258     0
27810    0
Name: sentiment, Length: 8299, dtype: object

In [26]:
y_train = y_train.astype('int8')

In [38]:
# Model building and training
model = LogisticRegression()
model.fit(X_train, y_train.values)

ValueError: Unknown label type: unknown. Maybe you are trying to fit a classifier, which expects discrete classes on a regression target with continuous values.

In [19]:
import dagshub

mlflow.set_tracking_uri("https://dagshub.com/param-unik/mlops-mini-project.mlflow")
dagshub.init(repo_owner="param-unik", repo_name="mlops-mini-project", mlflow=True)

mlflow.set_experiment("Logistic Regression Baseline")

Accessing as param-unik

Initialized MLflow to track repo "param-unik/mlops-mini-project"

Repository param-unik/mlops-mini-project initialized!

2024/08/10 18:07:02 INFO mlflow.tracking.fluent: Experiment with name 'Logistic Regression Baseline' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/5b978773af08411696575722bd01d2bf', creation_time=1723293423282, experiment_id='0', last_update_time=1723293423282, lifecycle_stage='active', name='Logistic Regression Baseline', tags={}>

In [20]:
with mlflow.start_run():

    # Log preprocessing parameters
    mlflow.log_param("vectorizer", "Bag of Words")
    mlflow.log_param("num_features", 5000)
    mlflow.log_param("test_size", 0.2)

    # Model building and training
    model = LogisticRegression()
    model.fit(X_train, y_train)

    # Log model parameters
    mlflow.log_param("model", "Logistic Regression")

    # Model evaluation
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)

    # Log evaluation metrics
    mlflow.log_metric("accuracy", accuracy)
    mlflow.log_metric("precision", precision)
    mlflow.log_metric("recall", recall)
    mlflow.log_metric("f1_score", f1)

    # Log model
    mlflow.sklearn.log_model(model, "model")

    # Save and log the notebook
    import os

    notebook_path = "exp1_baseline_model.ipynb"
    os.system(f"jupyter nbconvert --to notebook --execute --inplace {notebook_path}")
    mlflow.log_artifact(notebook_path)

    # Print the results for verification
    print(f"Accuracy: {accuracy}")
    print(f"Precision: {precision}")
    print(f"Recall: {recall}")
    print(f"F1 Score: {f1}")

2024/08/10 18:08:40 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.


Accuracy: 0.7942168674698795
Precision: 0.7816091954022989
Recall: 0.8039408866995074
F1 Score: 0.7926177756192326


2024/08/10 18:08:46 INFO mlflow.tracking._tracking_service.client: 🏃 View run auspicious-rook-575 at: https://dagshub.com/param-unik/mlops-mini-project.mlflow/#/experiments/0/runs/eccabde712fd456d9564208cc4f93c5d.
2024/08/10 18:08:46 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/param-unik/mlops-mini-project.mlflow/#/experiments/0.
